# Expected Credit Loss (ECL) Modelling and Validation
In this notebook we will focus on calculation of expected credit loss (ECL). ECL is calculated as the present value of the product of probability of default (PD), loss given default (LGD) and exposure at default (EAD) either over a one year horizon or a lifetime horizon depending on the weight of the risk. Apart from the three parameters we have also to include the forward looking perspective by including the economic scenarios in the calculation.

There are 3 staging allocation according to IFRS9 what is called the 'three-stage' model or general model for impairment depending on the significance increase of credit risk since initial recognition or at reporting date. The requirement is that a loan has to be grouped in either of the 3 stages to allow for the computaton of the ECL.

    * Stage 1 (One-year ECL) - these are financial instruments that have no significance increase in credit risk since initial recognition or whose credit risk is low at the reporting date. For these assets, 12 month ECL is recognised and the interest reenue is calculated on the gross carrying amount of this asset, that is, the credit allowance is not deducted.
    The 12 month ECL is the expected credit losses that result due to a default that has a probability of happening within 12 months after reporting date.

    * Stage 2 (Lifetime ECL) - these are instruments that have a significance increase in credit loss since initial regonition but do not have an objective evidence of impairement. The lifetime ECL is recognised and interest revenue is computed on the gross value of the asset.
    Lifetime ECL is the expected credit losses that are incurred upon default events over the lifetime of the asset.

    * Stage 3 (Impaired credits) - these are financial instruments that have an objective evidence of impairment at the reporting date. The lifetime ECL is recognised and the interest revenue is calculated on the net carrying amount, that is we deduct the credit allowance.

One must outline clearly on how to measure the significance increase in credit risk inorder to compare the risk of default occuring at initial recognition and the deafult risk occuring at the reporting date. There are 2 main ways to weigh this:
  * Quantitative indicator - Probability of default is the commonly used indicator to measure an increase in credit risk. A residual lifetime pd is used and in this case the same period of time remaining is considered for both pd at the reporting date and origination. A one year pd can be used if the change for it is a reasonble approximation for the lifetime change.
  * Qualitative indicator - this includes credit spread, credit default
swap price, market information related to the borrower, significant change in the credit rating.

1) In calculation of the one year ECL and lifetime ECL we use the one year pd and the lifetime pd respectively.

2) Loss Given Default (LGD) - this the proportion of non-recovered credit incase of default. To get the actual loss incurred we take a product of LGD and EAD.
$\textbf{LGD analysis}$ is conducted on $\textbf{defaulted accounts only}$. We calculate it as $$\text{LGD = (1 - recovery rate)}$$

$$\text{recovery rate} = \frac{\text{amount recovered}}{\text{total amount of the loan}}$$
- The LGD can also be calculated as follows: $$\textbf LGD_{l} = \textbf SEV_{wo,l} * (1 - \textbf p_{cu,l}) + \textbf SEV_{cu,l} * \textbf p_{cu,l}$$

1.   Probabilty of cure ($\textbf p_{cu}$) - this is probability of going back to the upto_date status.
2.   Severity - this can either be severity incase of write-off ($\textbf SEV_{wo,l}$) or severity incase of cure ($\textbf SEV_{cu,l}$)
3.   Time (discounting rate)
4.   Forward Looking property

3) Exposure at Default (EAD) - this is the predicted loss that a bank will incur incase of a default event. There are many methods of estimating it but we will focus on the Credit Conversion Factor (CCF) Method.
$$\text{EAD} = \text{current drawn amount} + \text{(CCF * undrawn amount)}$$

- CCF - this is a percentage that is set aside by regulators as an estimate of the likelihood of an off-balance item turnng into a credit loss in the bank's books.It is the proportion amount of currently undrawn amount that will likey to be drawn by the time of default.






In [1]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
# load the data
data=pd.read_csv("/content/loan_data_2007_2014.csv",low_memory=False)
pd.options.display.max_columns = None # does not limit the number of columns
print(data.shape)
data.head()

(466285, 75)


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,38.0,NaN,15.0,0.0,27783,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data["loan_status"].value_counts()

,count
loan_status,
Current,224226
Fully Paid,184739
Charged Off,42475
Late (31-120 days),6900
In Grace Period,3146
Does not meet the credit policy. Status:Fully Paid,1988
Late (16-30 days),1218
Default,832
Does not meet the credit policy. Status:Charged Off,761


In [ ]:
# We will work with only defaulted accounts ()
def_data=data[data["loan_status"].isin(["Charged Off","Default","Late (31-120 days)","Does not meet the credit policy. Status:Charged Off"])]
def_data.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.71,1008.71,456.46,435.17,0.0,117.08,1.1100,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,1071795,1306957,5600,5600,5600.0,60 months,21.28,152.39,F,F2,NaN,4 years,OWN,40000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I own a small h...,small_business,Expand Business & Buy Debt Portfolio,958xx,CA,5.55,0.0,Apr-04,2.0,NaN,NaN,11.0,0.0,5210,32.6,13.0,f,0.0,0.0,646.02,646.02,162.02,294.94,0.0,189.06,2.0900,Apr-12,152.39,NaN,Aug-12,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,1071570,1306721,5375,5375,5350.0,60 months,12.69,121.45,B,B5,Starbucks,< 1 year,RENT,15000.0,Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/16/11 > I'm trying to b...,other,Building my credit history.,774xx,TX,18.08,0.0,Sep-04,0.0,NaN,NaN,2.0,0.0,9279,36.5,3.0,f,0.0,0.0,1476.19,1469.34,673.48,533.42,0.0,269.29,2.5200,Nov-12,121.45,NaN,Mar-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,12,1064687,1298717,9000,9000,9000.0,36 months,13.49,305.38,C,C1,Va. Dept of Conservation/Recreation,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/15/11 > Plan to pay off...,debt_consolidation,freedom,245xx,VA,10.08,0.0,Apr-04,1.0,NaN,NaN,4.0,0.0,10452,91.7,9.0,f,0.0,0.0,2270.70,2270.70,1256.14,570.26,0.0,444.30,4.1600,Jul-12,305.38,NaN,Nov-12,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,14,1069057,1303503,10000,10000,10000.0,36 months,10.65,325.74,B,B2,SFMTA,3 years,RENT,100000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,other,Other Loan,951xx,CA,7.06,0.0,May-91,2.0,NaN,NaN,14.0,0.0,11997,55.5,29.0,f,0.0,0.0,7471.99,7471.99,5433.47,1393.42,0.0,645.10,6.3145,Oct-13,325.74,NaN,Mar-14,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Dependent variables
### Recovery Rate (RR)
- This is the proportion of the loan that the bank is able to recover incase there is a default. It ranges between 0 and 1.
- We use it as the $\textbf{dependent variable in the LGD modelling}$.

### Credit Conversion Factor (CCF)
- This is the dependent variable that we use when modelling for EAD.

In [ ]:
# Calculate the recovery rate
# To get all the recovered amount, we will add "total_rec_prncp", "total_rec_late_fee", "recoveries", "collection_recovery_fee"
def_data["recovery_rate"]=(def_data["total_rec_prncp"]+def_data["total_rec_late_fee"]+def_data["recoveries"]+def_data["collection_recovery_fee"])/def_data["funded_amnt"]

In [ ]:
# recovery rate is between 0 - 1 so any value above 1 will be equated to 1
def_data.loc[(def_data["recovery_rate"]>1),"recovery_rate"]=1

In [ ]:
# We will set a 0 to recovery rate=0 and 1 otherwise
def_data["rec_rate"]=np.where(def_data["recovery_rate"]==0,0,1)

In [ ]:
def_data["rec_rate"].value_counts()

,count
rec_rate,
1,50878
0,90


In [ ]:
# Calculate the CCF
# To be used in calculating the EAD
def_data["ccf"]=(def_data["funded_amnt"] - def_data["total_rec_prncp"]) / def_data["funded_amnt"]

In [ ]:
def_data["ccf"].describe()

,ccf
count,43236.000000
mean,0.735952
std,0.200742
min,0.000438
25%,0.632088
50%,0.789908
75%,0.888543
max,1.000000


In [ ]:
# split the data into train and test set
# will use the stratify parameter to make sure the minority class is equally distributed in both the test and train set
